# Transformers

This example illustrates the use of transformers with non-trivial phase shift and tap ratio.
The example is a copy of the [pandapower minimal example](https://github.com/e2nIEE/pandapower/blob/master/tutorials/minimal_example.ipynb).

In [8]:
import numpy as np
import pandas as pd

import pypsa

n = pypsa.Network()

n.add("Bus", "MV bus", v_nom=20, v_mag_pu_set=1.02)
n.add("Bus", "LV1 bus", v_nom=0.4)
n.add("Bus", "LV2 bus", v_nom=0.4)

n.add(
    "Transformer",
    "MV-LV trafo",
    type="0.4 MVA 20/0.4 kV",
    bus0="MV bus",
    bus1="LV1 bus",
)
n.add(
    "Line", "LV cable", type="NAYY 4x50 SE", bus0="LV1 bus", bus1="LV2 bus", length=0.1
)
n.add(
    "Generator", "External Grid", bus="MV bus", control="Slack", marginal_cost=10
)
n.add("Load", "LV load", bus="LV2 bus", p_set=0.1, q_set=0.05);

In [9]:
def run_power_flow(n: pypsa.Network) -> pd.DataFrame:
    n.lpf()
    n.pf(use_seed=True)
    return pd.DataFrame(
        {
            "Voltage Angles": n.buses_t.v_ang.loc["now"] * 180.0 / np.pi,
            "Voltage Magnitude": n.buses_t.v_mag_pu.loc["now"],
        }
    )

In [10]:
run_power_flow(n)

INFO:pypsa.network.power_flow:Performing linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x788827cb3a80> for snapshot(s) Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x788824526e70> for snapshots Index(['now'], dtype='object', name='snapshot')


,Voltage Angles,Voltage Magnitude
Bus,,
MV bus,0.000000,1.020000
LV1 bus,-150.760126,1.008843
LV2 bus,-149.884141,0.964431


In [11]:
n.transformers.tap_position = 2
run_power_flow(n)

INFO:pypsa.network.power_flow:Performing linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x788827e6e250> for snapshot(s) Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x788827e6e250> for snapshots Index(['now'], dtype='object', name='snapshot')


,Voltage Angles,Voltage Magnitude
Bus,,
MV bus,0.000000,1.020000
LV1 bus,-150.843911,0.959655
LV2 bus,-149.870837,0.912713


In [12]:
n.transformers.tap_position = -2
run_power_flow(n)

INFO:pypsa.network.power_flow:Performing linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x788827c95e50> for snapshot(s) Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x788827c95e50> for snapshots Index(['now'], dtype='object', name='snapshot')


,Voltage Angles,Voltage Magnitude
Bus,,
MV bus,0.000000,1.020000
LV1 bus,-150.681666,1.063133
LV2 bus,-149.896634,1.021202


Now play with tap changer on LV side

In [13]:
new_trafo_lv_tap = n.transformer_types.loc[["0.4 MVA 20/0.4 kV"]]
new_trafo_lv_tap.index = ["New trafo"]
new_trafo_lv_tap.tap_side = 1
new_trafo_lv_tap.T

,New trafo
f_nom,50.0
s_nom,0.4
v_nom_0,20.0
v_nom_1,0.4
vsc,6.0
vscr,1.425
pfe,1.35
i0,0.3375
phase_shift,150.0
tap_side,1


In [14]:
n.transformer_types = pd.concat([n.transformer_types, new_trafo_lv_tap])
n.transformers.type = "New trafo"
n.transformers.tap_position = 2
run_power_flow(n)

INFO:pypsa.network.power_flow:Performing linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x78882671dd60> for snapshot(s) Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x78882671dd60> for snapshots Index(['now'], dtype='object', name='snapshot')


,Voltage Angles,Voltage Magnitude
Bus,,
MV bus,0.000000,1.020000
LV1 bus,-150.755820,1.059317
LV2 bus,-149.964875,1.017221


In [15]:
n.transformers.T

Transformer,MV-LV trafo
bus0,MV bus
bus1,LV1 bus
type,New trafo
model,t
x,0.058283
r,0.01425
g,0.003375
b,-0.0
s_nom,0.4
s_nom_mod,0.0


In [16]:
n.transformers.tap_position = -2
run_power_flow(n)

INFO:pypsa.network.power_flow:Performing linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x7888267403d0> for snapshot(s) Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x7888267403d0> for snapshots Index(['now'], dtype='object', name='snapshot')


,Voltage Angles,Voltage Magnitude
Bus,,
MV bus,0.000000,1.020000
LV1 bus,-150.765232,0.958366
LV2 bus,-149.789394,0.911353


Finally, let's double-check that the phase shift is also there in the linear optimal power flow (optimisation) solution.

In [18]:
n.generators.p_nom = 1
n.lines.s_nom = 1
n.optimize(log_to_console=False)
pd.DataFrame(
    {
        "Voltage Angles": n.buses_t.v_ang.loc["now"] * 180.0 / np.pi,
        "Voltage Magnitude": n.buses_t.v_mag_pu.loc["now"],
    }
)

Index(['LV cable'], dtype='object', name='Line')
Index(['MV bus', 'LV1 bus', 'LV2 bus'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 3 primals, 9 duals
Objective: 1.00e+00
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper, Transformer-fix-s-lower, Transformer-fix-s-upper were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


,Voltage Angles,Voltage Magnitude
Bus,,
MV bus,0.627810,1.020000
LV1 bus,-0.165294,0.958366
LV2 bus,-0.462516,0.911353
